In [120]:
# Importar librerías para tratamiento de datos
import pandas as pd  

# Importar librerías para web scraping y manipulación de datos
from bs4 import BeautifulSoup
import requests
import re
import random
import os

# Importar librerías para automatización de navegadores web con Selenium
from selenium import webdriver  
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select  

# Importar librerías para pausar la ejecución
from time import sleep  

# Configuraciones
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

import warnings
warnings.filterwarnings('ignore')

In [121]:
rotten = pd.read_csv("../INFO_PELIS/peliculas_2021_2023.csv", index_col = 0)

In [122]:
lista_titulos= rotten["Titulo"].to_list()

In [123]:
lista_años= rotten["Año de estreno"].to_list()

In [124]:
list_busqueda_rotten = []
for titulo,año in zip(lista_titulos,lista_años):
    list_busqueda_rotten.append({titulo:año})

In [125]:
## Extraccion datos Rotten Tomatoes


url= "https://www.google.es/"


options = webdriver.FirefoxOptions()
options.page_load_strategy = "eager"
driver = webdriver.Firefox(options)

driver.get(url)

#Driver.maximize_window()

# Aceptar las cookies de GOOGLE
driver.find_element("css selector", "#L2AGLb > div").click()
sleep(1)


puntuacion_tomatoes = {}
contador = 10000
maximo = 15350
counter = contador
last_checkpoint = None

for peli in list_busqueda_rotten[contador:maximo]:
    for titulo,año in peli.items():
         
        busqueda = f"TOMATOMETER title: {titulo} - year: {año}"
        # print(busqueda)
        
        try:
                       
            buscador = driver.find_element("css selector",'#APjFqb')
            buscador.clear()
            buscador.send_keys(busqueda, Keys.ENTER)
            sleep(random.randint(1,3))

            try:
                puntuacion_tomatoes[titulo]=driver.find_element("css selector","#rso > div:first-child > div > div > div > div:nth-child(3) > div > span:nth-child(2)").text
                                
            except:
                puntuacion_completa= driver.find_element("css selector",'#rso > div:nth-child(1) > div > div > div:nth-child(3) > div').text
                puntuacion = re.search(r'\b(\d+)%', puntuacion_completa)
                puntuacion_tomatoes[titulo]=puntuacion.group()
        
        except:
            puntuacion_tomatoes[titulo]="No existe puntuacion"

        if counter % 100 == 0 and counter!= 0:
            print(f"Saving checkpoint... counter: {counter}")
            df_rotten = pd.Series(puntuacion_tomatoes)
            df_rotten.index.name="titulo"
            df_rotten.to_csv(f"cp_patri_rotten_{contador}_{counter}.csv")
            if last_checkpoint:
                os.remove(f"cp_patri_rotten_{contador}_{last_checkpoint}.csv")
            last_checkpoint = counter
            driver.close()
            driver = webdriver.Firefox(options)
            driver.get(url)
            # Aceptar las cookies de GOOGLE
            driver.find_element("css selector", "#L2AGLb > div").click()
            sleep(1)

        
        counter += 1


df_tomatoes = pd.Series(puntuacion_tomatoes,name="puntuacion")
df_tomatoes.index.name="titulo"
df_tomatoes.reset_index()
df_tomatoes.to_csv(f"{contador}_{maximo}_puntuacion_tomates_patri.csv", "w")
driver.close()

Saving checkpoint... counter: 10000
Saving checkpoint... counter: 10100
Saving checkpoint... counter: 10200
Saving checkpoint... counter: 10300
Saving checkpoint... counter: 10400
Saving checkpoint... counter: 10500
Saving checkpoint... counter: 10600
Saving checkpoint... counter: 10700
Saving checkpoint... counter: 10800
Saving checkpoint... counter: 10900
Saving checkpoint... counter: 11000
Saving checkpoint... counter: 11100
Saving checkpoint... counter: 11200
Saving checkpoint... counter: 11300
Saving checkpoint... counter: 11400
Saving checkpoint... counter: 11500
Saving checkpoint... counter: 11600
Saving checkpoint... counter: 11700
Saving checkpoint... counter: 11800
Saving checkpoint... counter: 11900
Saving checkpoint... counter: 12000
Saving checkpoint... counter: 12100
Saving checkpoint... counter: 12200
Saving checkpoint... counter: 12300
Saving checkpoint... counter: 12400
Saving checkpoint... counter: 12500
Saving checkpoint... counter: 12600
Saving checkpoint... counter